In [1]:
import numpy as np
from sklearn.datasets import fetch_openml

# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

# Convert labels to integers
# Normalize pixel values
X = X / 255.0


/home/valere/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [3]:
y = y.astype('int')

In [12]:
type(y)

pandas.core.series.Series

## Extraction Manuelle de Feature

In [13]:
import cv2

def extract_sift_features(X):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []
    
    # Reshape images to 28x28
    images = X.to_numpy().reshape(-1, 28, 28).astype(np.uint8)  # SIFT works with uint8 images
    
    for img in images:
        keypoints, descriptors = sift.detectAndCompute(img, None)
        keypoints_list.append(keypoints)
        if descriptors is not None:
            descriptors_list.append(descriptors)
        else:
            # If no descriptors are found, add an array of zeros
            descriptors_list.append(np.zeros((1, sift.descriptorSize()), dtype=np.float32))
    
    # Stack descriptors and handle varying lengths
    max_descriptors = max(len(d) for d in descriptors_list)
    descriptors_padded = [np.pad(d, ((0, max_descriptors - len(d)), (0, 0)), 'constant') for d in descriptors_list]
    
    return np.array(descriptors_padded).reshape(len(X), -1)

# Extract SIFT features from the dataset
X_sift_features = extract_sift_features(X)


In [17]:
X_sift_features[56]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

## Model Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sift_features, y, test_size=0.2, random_state=42)

# Train the model
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
